In [3]:
import pandas as pd
import numpy as np

In [4]:
submissions = ['resnet', 'keras_baseline']

In [5]:
submissions_dfs = []
for submission in submissions:
    submission_df = pd.read_csv("/mnt/extDisk/courses/data/statoil/data/" + submission + "/" + submission + ".csv")
    submission_df = submission_df.sort('id')
    submissions_dfs.append(submission_df)

/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  after removing the cwd from sys.path.


In [10]:
submissions_dfs = []
submission_df = pd.read_csv("/mnt/extDisk/courses/data/statoil/data/" + 'ensamble' + "/" + 'ensamble_0211' + ".csv")
submission_df = submission_df.sort('id')
submissions_dfs.append(submission_df)
submission_df = pd.read_csv("/mnt/extDisk/courses/data/statoil/data/" + 'pseudo_labeling' + "/" + 'pseudo_labeling_sure_resnet' + ".csv")
submission_df = submission_df.sort('id')
submissions_dfs.append(submission_df)

/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  This is separate from the ipykernel package so we can avoid doing imports until
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  


In [12]:
ensamble = submissions_dfs[0]
ensamble['is_iceberg'] = np.mean([submission.is_iceberg for submission in submissions_dfs],axis=0)

In [13]:
ensamble.head()

,id,is_iceberg
0,000c65d7,0.747903
1,001680af,0.541541
2,00193999,0.941407
3,00247e48,0.994865
4,00503faa,0.014434


In [14]:
ensamble.to_csv("/mnt/extDisk/courses/data/statoil/data/ensamble/ensamble_0211_resnet_pseudo.csv", index=False)

In [8]:
def mean_distance(df1,df2):
    print np.mean(np.absolute(df1['is_iceberg'] - df2['is_iceberg']))

In [11]:
mean_distance(submissions_dfs[0],submissions_dfs[1])

0.397544405602
